### EXERCISE 1.1.1. 

If unemployment compensation increases unemployment duration,
should we conclude that increasing such compensation is detrimental to society?
Provide some thoughts on this question in the context of the McCall model.

Answer:

Although it increases the unemployment duration, agents choose to be unemployed to maximize their utilities or EPV. Hence, from this perspective, it is not detrimental to the society.

In one extreme, if $c$ is very very high, it also possibly suggested this is a highly developed and automated world where people can have high unemployment compensation to enjoy instead of work.

### EXERCISE 1.1.2. 

The optimal action at time $t=0$ is determined by a time zero
reservation wage $w^*_0$ . The worker should accept the time zero wage offer if and only
if $w_0$ exceeds $w^*_0$ . Calculate $w^*_0$ for this problem, by analogy with $w^*_1$ in (1.4).


**Answer**

Let $h_0 = c+ \beta\sum_{w'\in W} v_1(w')\varphi(w')$ be the continuation value at time $t=0$.

Since the stopping value at $t=0$ is

$$
w_0 + \beta w_0 +\beta^2 w_0 = w_0(1+\beta +\beta^2)
$$

The reservation wage at $t=0$ is the wage equating the stopping value and the continuation value at time $t=0$, i.e.,

$$
w_0^*(1+\beta +\beta^2) = h_0
$$

This implies, 

$$
w_0^* = \frac{h_0}{1+\beta+\beta^2}
$$

### EXERCISE 1.1.3. 

Extend the above arguments to $T$ time periods, where $T$ can be
any finite number. Using Julia or another programming language, write a function
that takes $T$ as an argument and returns $(w^*_0,\ldots,w^*_T)$, the sequence of reservation
wages for each period.

First, we start with importing numpy for numerical operations and namedtuple to store the model parameters.

In [2]:
import numpy as np
from collections import namedtuple

A `namedtuple` is a convenient way to define a class. This type of data structure allows us to create tuple-like objects that have fields accessible by attribute lookup as well as being indexable and interable. 

In this model, we want to use the `namedtuple` to store values of the model, hence, we name it as `Model`. It requires the following parameters:

- `c`: the unemployment compensation
- `w_vals`: $W$, the finite wage space
- `n`: the cardinality of the wage space (in the following code, I use the uniform distribution, hence, this simplies the answer by not including extra parameters for the pdf)
- `β`: the discount factor

In [3]:
Model = namedtuple("Model", ("c", "w_vals", "n", "β","T"))

Then we use a function to input specific values into the `namedtuple`, i.e.,

In [4]:
def create_job_search_model(
    n = 50,          # wage grid size
    w_min = 11,      # lowest wage
    w_max = 60,      # highest wage
    c = 10,          # unemployment compensation
    β = 0.96,        # discount factor
    T = 10           # number of periods t= 0, 1,...,T
):
    """
    This function input the paramters with the above default values, and return a namedtuple
    """
    w_vals = np.linspace(w_min,w_max, n) # create a evenly spaced numbers over the specified intervals, with specified number of sample
    
    return Model(c = c, w_vals = w_vals, n = n, β = β, T=T) # return the namedtuple with the input parameters

Now we define a function that iteratively obtain the continuation value, and reservation wages

In [5]:
def reservation_wage(model):
    c, w_vals, n, β, T = model.c, model.w_vals, model.n, model.β, model.T  # Input the model parameters
    H = np.zeros(T+1)  # Initialize the continuation value sequence
    R = np.zeros(T+1)  # Initialize the reservation wage sequence
    S = np.zeros((T+1,n))  # Initialize the maximum values at each state at each period
    H[T] = c         # Input the last continuation value which is just the unemployment compensation
    R[T] = c         # The reservation wage at the last period is just the unemployment compensation
    S[T,:] = np.maximum(c, w_vals) # At period T, it is just comparing the unemployment compensation with the wages
    for t in range(1, T+1):
        H[T-t] = c + β * np.mean(S[T-t+1,:]) # Assuming uniform distribution, we only need to calculate the mean
        df = np.geomspace(1, β**t, t+1)   # this generate the sequence for the denominator
        dfs = np.sum(df)  # this is the denominator for the reservation wage calculation
        R[T-t] = H[T+1-t]/dfs    # This calculate the reservation wage at time T-t
        S[T-t,:] = np.maximum(dfs * w_vals, H[T-t])   # This returns the maximum values for each wage state by comparing the continuation value and stopping value
    return R
    

This function iteratively generate the reservation wage sequence. We can show the result by create the model and use this function to calculate the reservation wage sequence.

In [6]:
model = create_job_search_model()
reservation_wage(model)

array([36.50032766, 35.38365172, 34.07207926, 32.50704612, 30.6067209 ,
       28.23780776, 25.19318638, 21.10849802, 15.29705719,  5.10204082,
       10.        ])

### EXERCISE 1.2.1. 
Let $\alpha, s$ and $t$ be real numbers. Show that $\alpha \vee (s+t) \le s+\alpha \vee t$ whenever $s\ge 0$

**Proof**

Book-keeping:

$a\vee b$ means: $\max\{a,b\}$. Hence,

$$
\alpha \vee (s+t) \equiv \max\{\alpha, s+t\}
$$

and

$$
s + \alpha \vee t \equiv s+\max\{\alpha,t\}
$$

Suppose $\alpha \ge t$, this implies,

$$
s+\alpha \ge \alpha , s+\alpha \ge s+t
$$

Hence, the inequality holds.

Suppose $\alpha < t$, this implies, $\alpha < s+t$, hence the inequality also holds. This completes the proof.

### EXERCISE 1.2.2. 

Verify that the $\ell_1$ norm on $\mathbb{R}^n$ satisfies (a)–(d) above.

**Answer**

Book-keeping:

$\ell_1$ norm on $\mathbb{R}^n$ is defined as

$$
\|u\|_1 = \sum_{i=1}^n |u_i|
$$

Four conditions:

- [x] non-negativity: $\|u\|_1\ge 0$

$$
\|u\|_1 = \sum_{i=1}^n |u_i|\ge 0 
$$

- [x] Positive definiteness: $\|u\|_1 = 0 \iff u = 0$

$$
\|u\|_1 = 0 \iff u = 0
$$

- [x] Absolute homogeneity

$$
\|\alpha u\|_1 = \sum_{i=1}^n |\alpha u_i| = |\alpha|\sum_{i=1}^n |u_i| = \alpha \|u\|_1
$$

- [x] Triangle inequality

$$
\|u+v\|_1 = \sum_{i=1}^n |u_i+v_i| \le \sum_{i=1}^n|u_i|+|v_i| = \sum_{i=1}^n |u_i| + \sum_{i=1}^n|v_i| = \|u\|_1+\|v\|_1
$$


### EXERCISE 1.2.3. 

Fix $p\in \mathbb{R}^n$ with $p_i>0$ for all $i\in[n]$ and $\sum_{i}p_i=1$. 

Show that 

$$
\|u\|_{1,p} := \sum_{i=1}^n |u_i|p_i
$$

is a norm on $\mathbb{R}^n$.


**Answer**


Four conditions:

- [x] non-negativity: $\|u\|_{1,p}\ge 0$

$$
\|u\|_{1,p} = \sum_{i=1}^n |u_i|p_i\ge 0 
$$

- [x] Positive definiteness: $\|u\| = 0 \iff u = 0$

$$
\|u\|_{1,p} = 0 \iff u = 0
$$

- [x] Absolute homogeneity

$$
\|\alpha u\|_{1,p} = \sum_{i=1}^n |\alpha u_i|p_i = |\alpha|\sum_{i=1}^n |u_i|p_i = \alpha \|u\|_{1,p}
$$

- [x] Triangle inequality

$$
\|u+v\|_{1,p} = \sum_{i=1}^n |u_i+v_i|p_i \le \sum_{i=1}^n\left(|u_i|+|v_i|\right)p_i = \sum_{i=1}^n |u_i|p_i + \sum_{i=1}^n|v_i|p_i = \|u\|_{1,p}+\|v\|_{1,p}
$$

**Remark**

This implies expectation (absolute) is a norm

### EXERCISE 1.2.4

Show that the supremum norm is a norm on $\mathbb{R}^n$.

**Proof**

Book keeping:

Supremum norm of $u\in\mathbb{R}^n$:

$$
\|u\|_\infty = \max_{i}|u_i|
$$


Four conditions:

- [x] non-negativity: $\|u\|\ge 0$

$$
\|u\|_infty = \max|u_i|\ge 0  
$$

- [x] Positive definiteness: $\|u\| = 0 \iff u = 0$

$$
\|u\|_\infty = 0 \iff u = 0
$$

- [x] Absolute homogeneity

$$
\|\alpha u\|_\infty = \max|\alpha u_i| = |\alpha|\max|u_i|= |\alpha| \|u\|_{\infty}
$$

- [x] Triangle inequality

$$
\|u+v\|_\infty = \max|u_i+v_i| \le \max|u_i| + \max|u_i| = \|u\|_\infty + \|v\|_\infty
$$


### EXERCISE 1.2.5

Show that $\ell_0$ 'norm' 

$$
\|u\|_0 = \sum_{i=1}^n \mathbb{1}\{u_i\neq 0\}
$$

is not a norm.

**Proof**

It violates the absolute homogeneity condition. 

$$
\|\alpha u\|_0 = \sum_{i=1}^n \mathbb{1}\{\alpha u_i \neq 0\} =  \sum_{i=1}^n \mathbb{1}\{ u_i \neq 0\} = \|u\|_0\neq |\alpha|\|u\|_0
$$



### EXERCISE 1.2.6

Let's write $\|\cdot\|_a\sim \|\cdot\|_b$ if there exist finite $M,N$ such that

$$
M\|u\|_a\le \|u\|_b \le N\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

holds. Prove that $\sim$ is an equivalence relation on the set of all norms on $\mathbb{R}^n$.


**Proof**

Book Keeping:

Conditions of Equivalence relation:

- [x] Reflectivity: $\|\cdot \|_a \sim \|\cdot \|_a$

Take $M=N=1$. 

- [x] Symmetry: $\|\cdot\|_a \sim \|\cdot\|_b \implies \|\cdot\|_b\sim \|\cdot\|_a$

Suppose $\|\cdot\|_a \sim \|\cdot\|_b$, we have,

$$
M\|u\|_a\le \|u\|_b \le N\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

This implies 

$$
\frac{1}{N}\|u\|_b \le \|u\|_a\le \frac{1}{M} \|u\|_b \,\,\forall u\in\mathbb{R}^n
$$

where, $\frac{1}{N},\frac{1}{M}$ are finite (assume $M,N\neq 0$). Hence, $\|\cdot\|_b\sim \|\cdot \|_a$.


- [x] Transitivity: $\|\cdot\|_a\sim \|\cdot\|_b$, $\|\cdot\|_b\sim\|\cdot\|_c \implies \|\cdot\|_a\sim\|\cdot\|_c$

Suppose $\|\cdot\|_a\sim \|\cdot\|_b$ and $\|\cdot\|_b\sim\|\cdot\|_c$, then we have,

$$
M\|u\|_a\le \|u\|_b \le N\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

$$
M'\|u\|_b\le \|u\|_c \le N'\|u\|_b \,\,\forall u\in\mathbb{R}^n
$$


This implies 

$$
MM'\|u\|_a\le M'\|u\|_b \le \|u\|_c\le N'\|u\|_b \le NN'\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

Since $MM',NN'$ are also finite, hence, the transitivity is satisfied.

### EXERCISE 1.2.7 

Let $\|\cdot\|_a$ and $\|\cdot\|_b$ be any two norms on $\mathbb{R}^n$. Given a point $u$ in $\mathbb{R}^n$ and a sequence $(u_m)$ in $\mathbb{R}^n$. Use

$$
M\|u\|_a\le \|u\|_b \le N\|u\|_a \,\,\forall u\in\mathbb{R}^n
$$

to confirm that $\|u_m-u\|_a\to 0\implies \|u_m-u\|\to 0$ as $m\to \infty$.


**Proof**

Suppose $\|u_m-u\|_a\to 0$ as $m\to\infty$. Hence, for all $\epsilon>0$, there exists $M(\epsilon)\in \mathbb{N}$ such that, $m\ge M$ implies 

$$
\|u_m-u\|_a <\frac{\epsilon}{N}
$$

this implies, by the given equivalence, we have

$$
\|u_m-u\|_b\le N\|u_m-u\|_a<\epsilon
$$

Hence, we have $\|u_m-u\|_b\to 0$ as $m\to0$.

### EXERCISE 1.2.8 

Let $\|\cdot\|$ be any norm on $\mathbb{R}^n$. Fixing a point $u\in\mathbb{R}^n$ and a sequence $(u_m)$ in $\mathbb{R}^n$.

Let $u^i$ and $u^i_m$ be the $i-th$ component of $u$ and $u_m$ respectively. 

Show that $u_m^i\to u^i$ for all $i\in \{1,2,\ldots,n\}$ if and only if $\|u_m-u\|\to 0$.

**Proof**

$(\implies)$

Suppose $u_m^i \to u^i$ for all $i\in\{1,2,\ldots,n\}$. This implies for all $\epsilon>0$, there exist $M^i(\epsilon)\in\mathbb{N}$ such that, $m^i\ge M^i$ implies 

$$
|u_m^i -u^i|<\frac{\epsilon}{n}
$$

for all $i=1,2,\ldots,n$.

Take $M = \max\{M^1,M^2,\ldots,M^n\}$. $M$ exists as $n$ is finite. Then, we have for all $\epsilon>0$, there exist $M$ as defined above, such that $m^i\ge M$, we have

$$
\|u_m-u\| = \sum_{i=1}^n |u_m^i-u^i|< n\frac{\epsilon}{n} = \epsilon
$$

Hence, $\|u_m-u\|\to 0$ as $m\to 0$.


$(\impliedby)$

Suppose $\|u_m-u\|\to 0$ as $m\to 0$. This means for all $\epsilon>0$, there exists $M(\epsilon)>0$ such that $m\ge M$ implies 

$$
|u_m^i-u^i|\le \|u_m-u\| = \sum_{i}^n |u_m^i-u^i|<\epsilon
$$

for all $i=1,2,\ldots, n$.

This completes the proof.


### EXERCISE 1.2.9 

Show that the operator $\|\cdot \|_o$ is submultiplicative on $\mathbb{R}^{n\times n}$.

Provide a counterexample to claim that $\|\cdot\|_\infty$ is submultiplicative.


**Proof**

Book Keeping:

**Submultiplicative**:

For all $A, B \in\mathbb{R}^{n\times n}$,

$$
\|AB\|\le \|A\|\|B\|
$$

Definition of operator $\|\cdot\|_o$:

$$
\|B\|_o:= \max_{\|u\|=1}\|Bu\|
$$

where $B\in\mathbb{R}^{n\times n}, u\in\mathbb{R}^n$

From the definition, we get,

$$
\|AB\|_o = \max_{\|u\|=1}\|ABu\|
$$

We have the $i,j$ element of $AB$ is 

$$
(AB)_{ij} = \sum_{k=1}^n a_{ik}b_{kj}
$$

and hence, we have the $i$th element of $ABu$ is 

$$
(ABu)_i = \sum_{j=1}^n (AB)_{ij} u_j = \sum_{j=1}^n \sum_{k=1}^n a_{ik} b_{kj} u_j
$$

Hence, let $u^a, u^b, u^{ab}$ be the unit vector that maximize the norm, we have

\begin{align*}
\|AB\|_o &=  \left(\sum_{i=1}^n\left(\sum_{j=1}^n \sum_{k=1}^n a_{ik} b_{kj} u^c_j\right)^2\right)^{1/n}
\end{align*}

Since all finite dimension Euclidean norm is equivalent, we only need to show the required inequality in terms of $\ell_1$ norm, i.e., 

$$
\sum_{i=1}^n\left|\sum_{j=1}^n \sum_{k=1}^n a_{ik} b_{kj} u^c_j\right|\le \left(\sum_{i=1}^n\left|\sum_{j=1}^n a_{ij}u^a_{j}\right|\right)\left(\sum_{i=1}^n\left|\sum_{j=1}^n b_{ij}u^b_{j}\right|\right)\\
$$
$$
= \left(\left|\sum_{j=1}^n a_{1j}u^a_{j}\right| +\ldots+\left|\sum_{j=1}^n a_{nj}u^a_{j}\right|\right)\left(\left|\sum_{j=1}^n b_{1j}u^b_{j}\right|+\ldots+\left|\sum_{j=1}^n b_{nj}u^b_{j}\right|\right)
$$


Book keeping

The definition of $\|\cdot\|_\infty$ is equal to the maximum of the absolute value of the elements, i.e.,

$$
\|A\|_\infty= \max_{1\le i,j\le n} |A_{ij}|
$$

We want to find a counter-example to show that $\|\cdot\|_{\infty}$ is not submultiplicative, i.e., we want to find $A,B$ such that

$$
\|AB\|_{\infty} > \|A\|_\infty\|B\|_\infty
$$

Let $A = 
\begin{bmatrix}
1 & 2\\
0 & 1\\
\end{bmatrix}$ and $B = 
\begin{bmatrix}
3 & 0 \\
4 &0\\
\end{bmatrix}$, then we have $AB =
\begin{bmatrix}
11 & 0\\
4  & 0\\
\end{bmatrix}$

Hence, we have $\|AB\|_\infty = 11> \|A\|_{\infty} \|B\|_{\infty} = 2 \times 4 = 8$



### EXERCISE 1.2.10 

Prove that 

$$
\rho(\alpha B) = |\alpha| \rho(B),\,\,\forall \alpha \in \mathbb{R}
$$

**Proof**

Book-keeping

$\rho(B)$ is the spectral radius of $B$, i.e.,

$$
\rho(B) = \max |\lambda^B|
$$

Let $\lambda_i$ be the eigenvalues of $B$. Then, we have by the definition of eigenvalues,

$$
Bv_i =\lambda_iv_i
$$

This implies

$$
\alpha Bv_i = \alpha \lambda_i v_i
$$

Hence, $\alpha \lambda_i$ are the eigenvalues of $\alpha B$. Since we scale all the eigenvalues by the same constant $\alpha$. This implies $\max |\alpha\lambda_i| =\max|\alpha| |\lambda_i|$. 

Hence, this proves the statement.

### Exericse 1.2.11 

Use Lemma 1.2.2 to show that 

1. $\|B^k\|\to 0$ as $k\to \infty$ $\iff$ $\rho(B)<1$

2. $\rho(B)>1 \implies \|B^k\|\to \infty$ as $k\to\infty$

**Proof**

Book keeping: Lemma 1.2.2

If $B$ is any square matrix and $\|\cdot\|$ is any matrix norm, then 

$$
\rho(B)^k\le \|B\|^k,\,\,\forall k\in\mathbb{N}
$$

and the Gelfand's formula

$$
\|B^k\|^{1/k} \to\rho(B)
$$

as $k\to\infty$



Part 1 of the proof:

$(\implies)$

Suppose $\|B^k\|\to 0$ as $k\to\infty$. This implies,

$$
\|B^k\|^{1/k}\to 0,\,\,k\to\infty
$$

By the lemma 1.2.2 and Gelfand's formula, this implies

$$
\rho(B)^k \to 0,\,\,k\to\infty
$$

Hence, this implies $\rho(B)<1$.

$(\impliedby)$

Suppose $\rho(B)<1$, we have by the Gelfand's formula,

$$
\|B^k\|^{1/k} \to\rho(B)
$$

This implies, for all $\epsilon>0$, there exists $K(\epsilon)\in\mathbb{N}$ such that for all $k\ge K$, we have,

$$
|\|B^k\|^{1/k}-\rho(B)|<\epsilon
$$

As $\rho(B)<1$, this implies, $\|B^k\|^{1/k}<1$ as $k\to\infty$. We have, as $k\to \infty$

$$
\|B^k\| = \left(\|B^k\|^{1/k}\right)^k \to 0
$$

This proves part 1.


**Part 2 proof**:

Suppose $\rho(B)>1$. As $k\to \infty$, we have 

$$
\|B^k\|\ge \rho(B)^k \to \infty
$$

This completes the proof.

### EXERCISE 1.2.12.

Prove if $A$ and $B$ are square matrices that commute i.e., $AB=BA$, then

$$
\rho(AB)\le \rho(A)\rho(B)
$$

Hint: Show $(AB)^k = A^kB^k$ and use the Gelfand's formula

**Proof**

We follow the hint. 

Suppose $[A,B]=0$. We have
\begin{align*}
(AB)^k = ABABABAB\ldots AB = AA\ldots AABB\ldots BB = A^k B^k
\end{align*}

Using Gelfand's formula, we have

$$
\|(AB)^k\|^{1/k} \to \rho(B)
$$

This implies, for all $\epsilon>0$, there exist $K^{ab}(\epsilon)\in\mathbb{N}$ such that $k\ge K^{ab}$ implies 

$$
\|(AB)^k\|^{1/k} - \rho(AB)<\epsilon
$$

Similarly, we have,

For all $\epsilon>0$, there exist $K^{a}(\epsilon)\in\mathbb{N}$ such that $k\ge K^{a}$ implies 

$$
\|A^k\|^{1/k} - \rho(A)<\epsilon
$$

For all $\epsilon>0$, there exist $K^{b}(\epsilon)\in\mathbb{N}$ such that $k\ge K^{b}$ implies 

$$
\|B^k\|^{1/k} - \rho(B)<\epsilon
$$

Let $K = \max\{K^{ab}, K^a,K^b\}$, we have,

For all $\epsilon>0$, there exist $K(\epsilon)\in\mathbb{N}$ such that $k\ge K$ implies 

$$
\|A^k\|^{1/k} - \rho(A)<\epsilon
$$

$$
\|B^k\|^{1/k} - \rho(B)<\epsilon
$$

$$
\|(AB)^k\|^{1/k} - \rho(AB)<\epsilon
$$

We have

$$
\rho(AB)\le \|(AB)^k\|^{1/k}\le \|A^k\|^{1/k}\|B^k\|^{1/k}<\epsilon^2 + \epsilon\rho(A) + \epsilon\rho(B) + \rho(A)\rho(B)
$$

Since, this is true for all $\epsilon$, we have

$$
\rho(AB)\le \rho(A)\rho(B)
$$

as required.

### EXERCISE 1.2.13

Prove $\rho(A)<1$ implies that the series $\sum_{k\ge 0}A^k$ converges, in the sense that every element of the matrix

$$
S_K:= \sum_{k=0}^K A^k
$$

converges as $K\to \infty$.

**Proof**

Let $(\lambda,v)$ be a eigen pair of $A$. We have $|\lambda|<1$. Then we have,

$$
S_Kv = Iv + Av+A^2v + \ldots + A^Kv = v + \lambda v+ \lambda^2v+\ldots \lambda^K v
$$

Hence, since $|\lambda|<1$, we have,

$$
S_\infty v = \frac{1}{1-\lambda}v
$$

Hence, this implies $S_K$ converges.

### EXERCISE 1.2.14

Show that  when $\sum_{k\ge 0}A^k$ exists, the inverse of $I-A$ exists and indeed

$$
(I-A)^{-1} = \sum_{k\ge 0} A^k
$$

**Proof**

We show this by showing 

$$
(I-A)\sum_{k\ge 0}A^k = I
$$

We have 

$$
(I-A)(I+A + A^2 + \ldots) = (I+A+A^2\ldots)- (A+A^2+\ldots) = I
$$

### EXERCISE 1.2.15

Let $U$ be any set and let $T$ be a self-map on $U$. Suppose there exists an $\overline u\in U$ and an $m\in\mathbb{N}$ such that $T^ku = \overline u$ for all $u\in U$ and $k\ge m$. 

Prove that under this condition, $\overline u$ is the unique fixed point of $T$ in $U$.

**Proof**

We first show that $\overline u$ is a fixed point. Then we show that it is unique.


First, we show that $\overline u$ is a fixed point.

Let $u\in U$ be any element of $U$. We have 

$$
T^{m+1} u = \overline u = T\circ T^m(u) = T(\overline u)
$$

By the definition, $\overline u$ is a fixed point of $T$ in $U$.


Then, we prove this fixed point is unique.

Suppose $v\in U$ is any other fixed point of $T$ in $U$. Then we have

$$
T^mv = v = \overline u
$$

Hence, we have $\overline u$ is the unique fixed point.


### EXERCISE 1.2.16

Let $T$ be a self-map on $U\subset \mathbb{R}^d$.

Prove that 

If $T^mu\to u^*$ as $m\to\infty$ for some pair $(u,u^*)\in U$ and, in addition, $T$ is continuous at $u^*$, then $u^*$ is a fixed point of $T$.

**Proof**

By continuity, we have $T^mu \to T^m u^*$. Hence, this implies $T^k u^*=u^*$ for all $k\ge m$. Hence let $k=m+1$, we have $T\circ T^mu^*= Tu^*=u^*$.

Hence, $u^*$ is a fixed point of $T$ in $U$.


**Another proof**

Let $u_m:= T^m u$ for all $m\in\mathbb{N}$.

By continuity and $u_m\to u^*$, we have $Tu_m\to Tu^*$.

Since $Tu_m\to u^*$ and limit is unique, we have $Tu^*=u^*$, hence $u^*$ is a fixed point.

### EXERCISE 1.2.17

Let $U= \mathbb{R}^n$ and let $T$ be defined by $Tu = Au+b$, using induction, prove that 

$$
T^k u = A^ku + A^{k-1}b + \cdots+ Ab +b
$$

for all $u\in U$ and $k\in\mathbb{N}$

And then, show that $T$ is globally stable on $U$ whenever $\rho(A)<1$.

**Proof Part 1**

$k=1$, we have,

$$
Tu = Au + b
$$

$k=2$, we have,

$$
T^2u = T(Au+b) = A(Au+b)+b = A^2u + Ab +b
$$

Keep iterating, we have

$$
T^k u = A^ku + A^{k-1}b + \cdots+ Ab +b
$$

as required.


**Proof Part 2**

Book keeping:

$T:U\mapsto U$ is globally stable on $U$ if $T$ has a unique fixed point $u^*$ in $U$ and

$$
T^ku \to u^*,\,\,k\to\infty
$$

for all $u\in U$.

Suppose $\rho(A)<1$.

From part 1, we have,

$$
T^ku = A^ku + \left(\sum_{i=1}^{k-1}A^k\right)b
$$

As $k\to \infty$ we have, $\|A^k\|\to 0 implies A^k=0$ as $k\to\infty$ by the positive definiteness of the matrix norm.

Then, we let $u_k:=T^ku$. This gives, by the previous results

$u_k \to \left(\sum_{k\ge 0}A^k\right)b  = (I-A)^{-1}b$

as $k\to infty$.

This is true for all $u\in U$. Let $u^*= (I-A)^{-1}b$. This implies, $u^*$ is a unique fixed point in $U$ for all $u\in U$ by previous exercises.

Hence, $T$ is globally stable.


### EXERCISE 1.2.18

Let $T$ be a globally stable self-map on $U\subset\mathbb{R}^n$, with fixed point $u^*$.

Prove that if $C$ is closed and $T$ is invariant on $C$, then $u^*\in C$.

**Proof**

Let $u\in C$ be any point in $C$. Let $u_k:= Tu$ for $k\in\mathbb{N}$. 

As $T$ is globally stable. We have, $u_k\to u^*$. 

As $T$ is invariant on $C$, $\{u_k\}_{k\in \mathbb{N}}\subset C$,i.e., this sequence is in the closed set $C$. By the definition of closed set, all limit points are in the closed set.

Hence, we have $u^*\in C$.

### EXERCISE 1.2.19

Let $T$ be a contraction on $U$ with respect to a norm $\|\cdot\|$. 

Show that $T$ is continuous on $U$ and has at most one fixed point in $U$.

**Proof**

Book keeping:

$T:U\mapsto V$ is called **continous** on $U$ if 

$$
(u_m)\subset U, u_m\to u\implies Tu_m\to T_u
$$

at every $u\in U$.

Contraction:

$T$ is called a contraction on $U$ with respect to $\|\cdot\|$ if 

$$
\exists \lambda<1, \|Tu-Tv\|\le \lambda\|u-v\|
$$

for all $u,v\in U$.

Let $(u_m)\subset U$ such that $u_m\to u$, i.e., for all $\epsilon>0$, there exists $M(\epsilon)\in\mathbb{N}$ such that $m\ge M$ implies $\|u_m-u\|<\epsilon$.

Since $T$ is a contraction, we have

$$
\|Tu_m-Tu\|< \|u_m-u\|<\epsilon
$$

Hence, we have $Tu_m\to Tu$. Hence, $T$ is continuous on $U$.

**Proof part 2**

We prove that $T$ has at most one fixed point in $U$ by contradiction.

Suppose there are two or more fixed points. Take $u^*, v^*$ be any arbitrary two fixed points. 

By the definition of fixed point, we have

$$
Tu^*=u^*, Tv^*=v^*
$$

Then, by the definition of contraction, we have

$$
\|Tu^*-Tv^*\|=\|u^*-v^*\|<\|u^*-v^*\|
$$

This contradiction completes the proof.

### EXERCISE 1.2.20

Let $U=\mathbb{R}^n$ and let $Tx = Ax+b$, there $A$ is $n\times n$ and $b$ is $n\times 1$. 

Prove that $T$ is a contraction of modulus $\|A\|$ on $U$ whenever $\|A\|<1$.

**Proof**

Book keeping:

$\|A\|= \max_{\|u\|=1} \|Au\|$

Suppose $\|A\|<1$. Let $u,v\in U=\mathbb{R}^n$. We have,

$$
\|Tu-Tv\| = \|Au+b-(Av+b)\| = \|Au-Av\| = \|A(u-v)\|
$$

Let $\overline{(u-v)}$ denote the unit vector in the direction of $u-v$, i.e.,

$$
u-v = \|u-v\|\overline{(u-v)}
$$

By the absolute homogeneity of Euclidean norm, we have,

$$
\|Tu-Tv\|= \|A\|u-v\|\overline{(u-v)}\|= \|u-v\|\|A\overline{(u-v)}\|
$$

Since we have $\|A\|<1$, this implies $\|A\overline{(u-v)}\|<1$. This gives,

$$
\|Tu-Tv\|= \|u-v\|\|A\overline{(u-v)}\|< \|u-v\|
$$

Hence, $T$ is a contraction.

### EXERCISE 1.2.21

Let $U$ and $T$ have the properties stated in Theorem 1.2.3.

Fix $u_0\in U$ and let $u_m:= T^m u_0$. 

Show that

$$
\|u_m-u_k\|\le \sum_{i=m}^{k-1}\lambda^i\|u_0-u_1\|
$$

holds for all $m,k\in\mathbb{N}$

**Proof**

Bookkeeping:

**Theorem 1.2.3. (Banach's contraction mapping theorem)**

If $U$ is closed in $\mathbb{R}^n$ and $T$ is a contraction of modulus $\lambda$ on $U$ with respect to some norm $\|\cdot\|$ on $\mathbb{R}^n$, then $T$ has a unique fixed point $u^*\in U$ and 

$$
\|T^ku-u^*\|\le \lambda^k\|u-u^*\|
$$

for all $k\in\mathbb{N}, u\in U$.

In particular, $T$ is globally stable on $U$. 


Now fix $u_0\in U$ and let $u_m:= T^m u_0$. W.l.o.g., we assume $m<k$. (This state is trivial when $m=k$).

We have,

\begin{align*}
\|u_m-u_k\| &= \|(u_m-u_{m+1})+(u_{m+1}-u_{m+2})-\cdots+(u_{k-1}-u_{k})\|\\
&\le \|u_m-u_{m+1}\|+\cdots+\|u_{k-1}-u_k\|\tag{Triangle inequality}\\
&= \|Tu_{m-1}-Tu_m\|+\cdots+\|Tu_{k-2}-Tu_{k-1}\|\\
&= \sum_{i=m}^{k-1}\|Tu_{i-1}-Tu_i\|\\
&\le \sum_{i=m}^{k-1}\lambda\|u_{i-1}-u_i\|\tag{Theorem}\\
\end{align*}

We can keep iterating $i-1$ times, this gives,

$$
\|u_m-u_k\|\le \sum_{i=m}^{k-1}\lambda^i\|u_{0}-u_1\|
$$

### EXERCISE 1.2.22. 

Using the result in Exercise 1.2.21.

Prove that $(u_m)$ is a Cauchy sequence in $\mathbb{R}^n$.

**Proof**

Bookkeeping:

A sequence $(u_m)\subset\mathbb{R}^n$ is called Cauchy sequence if,

for any $\epsilon>0$, there exists $N(\epsilon)\in\mathbb{N}$ such that for all

$$
m,k\ge N\implies \|u_m-u_n\|<\epsilon
$$


Let $\ell:= \|u_0-u_1\|$. For any $m,k\in \mathbb{N}$, we have, from Exercise 1.2.21,

$$
\|u_m-u_k\|\le \sum_{i=m}^{k-1}\lambda^i\ell = \sum_{i=0}^{k-1-m}\lambda^{m+i}\ell\le \sum_{i=0}^\infty\lambda^{m+i}\ell
$$

Since $\lambda<1$, we have,

$$
\|u_m-u_k\|\le \dfrac{1}{1-\lambda^m}\ell
$$

To solve $N(\epsilon)\in\mathbb{N}$, we solve

$$
\dfrac{1}{1-\lambda^N}\ell<\epsilon \implies \lambda^N>\ell/\varepsilon+1 \implies N>\log_\lambda (\ell/\varepsilon+1)
$$

Hence, under this construction, for all $m,k\ge N$, we have

$$
\|u_m-u_k\|<\epsilon
$$

This implie $(u_m)$ is a Cauchy sequence in $U$.

### EXERCISE 1.2.23

Prove that $u^*\in U$.

**Proof**

Since $(u_m)\mathbb{R}^n$ is a Cauchy sequence and $\mathbb{R}^n$ is complete. This imples this sequence has a limit point $u^*\in\mathbb{R}^n$. 

Since $(u_m)\subset U$ and $U$ is closed, by the definition of closed set (contains every limit point), $u^*\in U$.

### EXERCISE 1.2.24

Let $T$ be a contraction of modulus $\beta$ on $\mathbb{R}^n$ with fixed point $\overline u$.

Consider the damped or relaxed iteration scheme

$$
u_{n+1} = (1-\alpha)u_n + \alpha Tu_n
$$

Show that, for any choice of $u_0$, these iterates converge to $\overline u$ whenever $0<\alpha\le 1$.

**Proof**

Let $u_0\in \mathbb{R}^n$ be any point in $\mathbb{R}^n$. We want to show that for all $\epsilon>0$, there exists $N(\epsilon)\in\mathbb{N}$ such that 

$$
n\ge N \implies \|u_n-\overline u\|<\epsilon
$$

\begin{align*}
\|u_n-\overline u\| &= \|(1-\alpha)u_{n-1}+\alpha Tu_{n} -\overline u\|= \|(1-\alpha)(u_{n-1}-\overline u) + \alpha (Tu_n-\overline u)\|\\
&\le (1-\alpha)\|u_{n-1}-\overline u\|+\alpha \|Tu_n-\overline u\|\\
&= (1-\alpha)\|u_{n-1}-\overline u\| + \alpha \|Tu_n -T\overline u\|\\
&\le (1-\alpha)\|u_{n-1}-\overline u\| + \alpha\beta \|u_n -\overline u\|
\end{align*}

This implies,

$$
\|u_n - \overline u\|\le \frac{1-\alpha}{1-\alpha\beta}\|u_{n-1}-\overline u\|\le \left(\frac{1-\alpha}{1-\alpha\beta}\right)^n\|u_{0}-\overline u\|
$$

Since we have $\left(\frac{1-\alpha}{1-\alpha\beta}\right)<1$ and $\|u_0-\overline u\|<\infty$ for all $u_0\in \mathbb{R}^n$, we can use $N(\epsilon)\in\mathbb{N}$ such that 

$$
\|u_n - \overline u\|\le \left(\frac{1-\alpha}{1-\alpha\beta}\right)^n\|u_{0}-\overline u\|\le\left(\frac{1-\alpha}{1-\alpha\beta}\right)^N\|u_{0}-\overline u\|<\epsilon
$$

This completes the proof.

### EXERCISE 1.2.25



Let $f(k) = Ak^\alpha$ with $A>0$ and $0<\alpha<1$.

Show that, while the Solow-Swan map $g(k) = sAk^\alpha + (1-\delta)k$ sends $U:=(0,\infty)$ to itself, $g$ is not a contraction on $U$.

Hint: use the definition of the derivative of $g$ as a limit and consider the derivative $g'(k)$ for $k$ close to zero.

**Proof by contradiction**

Following the hint, we use the definition of the derivative of $g$, i.e.,

$$
g'(k) = \lim_{n\to \infty}\frac{g(k+1/n)-g(k)}{1/n} 
$$

Suppose $g$ is a contraction with modulus of contraction $\lambda$. Since $g$ is increasing with $k$, we have

$$
g'(k) = \lim_{n\to\infty}\frac{g(k+1/n)-g(k)}{1/n} \le\lim_{n\to\infty}\frac{\lambda(k+1/n-k)}{1/n}=\lambda
$$

for all $k\in(0,\infty)$.

However, we know $g'(k)\to\infty$ as $k\to 0$. Since $g$ is continuous, this implies there exists $k$ such tht $g'(k)>\lambda$.

This contradiction gives the proof. 

### EXERCISE 1.2.26

Show that in the setting of Exercise 1.2.25, the unique fixed point of $g$ in $U$ is

$$
k^*:= \left(\frac{sA}{\delta}\right)^{1/(1-\alpha)}
$$

Prove that for $k\in U$,

1. $k\le k^*\implies k\le g(k)\le k^*$

2. $k^*\le k\implies k^*\le g(k)\le k$

And conclude that $g$ is globally stable on $U$. Give Reason.

**Proof**

For the fixed point $k^*$, we have,

$$
g(k^*) = sA(k^*)^\alpha + (1-\delta)k^*=k^*
$$

Hence, we have, 

$$
\delta k^* = sA(k^*)^\alpha
$$

$$
k^* = \left(\frac{sA}{\delta}\right)^{1/(1-\alpha)}
$$


**Proof part 1**

Suppose $k\le k^*$. Since $g$ is increasing with $k$, we have,

$$
g(k)\le g(k^*)=k^*
$$

Moreover, since $g$ is concave, this implies for all $k\in(0,k^*)$, $k\le g(k)$.

This completes the proof for part 1.

**Proof part 2**

Suppose $k^*\le k$. Since $g$ is increasing with $k$, we have,

$$
k^*=g(k^*)\le g(k)
$$

Similarly, we have $k>k^*\implies k\ge g(k)$. This completes for proof of part 2.

**Conclusion**

From the above two parts, we find for $k\le k^*$, we have $k\le g(k)$. Let $k_n := g^n(k)$ Hence iteratively applying $g$ will have

$$
k\le g(k)\le g(g(k))\le \cdots\le k^*
$$

or

$$
k\le k_1\le k_2\le\cdots\le k^*
$$

This is true for any $k\le k^*$. This implies $k\uparrow k^*$ for all $k\le k^*$.

Similarly, we can show that for all $k\ge k^*$, $k\downarrow k^*$.

Hence, $g$ is globally stable on $U$.

### EXERCISE 1.2.27

Fix $h\in\mathbb{R}^X$. Show that $\varphi^*\in\arg\max_{\varphi\in\mathcal{D}(X)}\langle h,\varphi\rangle$ if and only if $\varphi^*$ is supported on $\arg\max_{x\in X} h(x)$.

**Proof**

Bookkeeping:

We say $\varphi$ is supported on $X_0$ if $\varphi(x)>0$ implies $x\in X_0$.



$(\implies)$

Suppose $\varphi^*\in\arg\max_{\varphi\in\mathcal{D}(X)}\langle h,\varphi\rangle$. Let $X_0:= \arg\max_{x\in X} h(x)$. Let $x^*\in X_0$ and $h(x^*):= \max_{x\in X}h(x)$.

Suppose $\varphi^*(x)>0$ and $x\not\in X_0$. This implies there exists $\varphi'$ such that 

$\varphi'(x^*)=1, \varphi'(x)=0$

This gives,

$$
\langle h,\varphi'\rangle = h(x^*)>\langle h,\varphi^*\rangle
$$

as $h(x)<h(x^*)$.

This implies $\varphi^*\not\in\arg\max_{\varphi\in\mathcal{D}(X)}\langle h,\varphi\rangle$. This contradiction gives the proof.

$(\impliedby)$

Suppose $\varphi^*$ is supported on $X_0$. This implies 

$$
\langle h, \varphi^*\rangle = h(x^*)
$$

This gives the proof directly.

### EXERCISE 1.2.28

Prove that the quantile function is additive over constants.

That is, for any $\tau\in[0,1]$, random variable $X\in \mathcal{X}$ and $\alpha\in\mathbb{R}$, we have

$$
Q_\tau(X+\alpha) =Q_\tau(X)+\alpha
$$

**Proof**

Bookkeeping.

The $\tau$-th quantile of $X$ is

$$
Q_\tau X: =\min\{x\in\mathcal{X}: \Phi(x)\ge\tau\}
$$

where,

$$
\Phi(x):= \mathbb{P}\{X\le x\} = \sum_{x'\in \mathcal{X}}\mathbb{1}\{x'\le x\}\varphi(x')
$$

Let $Y:=X+\alpha$, then $y=x+\alpha$.

By the definition of cumulative distribution function, we have,

$$
\Phi(y)= \mathbb{P}\{Y\le y\} = \mathbb{P}\{X+\alpha\le x+\alpha\} = \mathbb{P}\{X\le\alpha\} = \Phi(x)
$$

Then use the definition of quantile function, we have,

$$
Q_\tau Y =\min\{y\in\mathcal{X}: \Phi(y)\ge\tau\} = \min\{x+\alpha\in\mathcal{X}:\Phi(x)\ge\tau\} = \min\{x\in\mathcal{X}:\Phi(x)\ge tau\} + \alpha
$$

This gives the proof.

### EXERCISE 1.3.1

Verify that 

$$
|\alpha\vee x-\alpha \vee y|\le |x-y|,\,\,(\alpha,x,y\in\mathbb{R})
$$

always holds.

**Proof**

W.l.o.g., we assume $x<y$. ($x=y$ case is trivial).

Suppose $\alpha\le x<y$, we have $LHS = |x-y|=RHS$.

Suppose $x<\alpha\le y$, we have $LHS = |\alpha -y| \le |x-y|$.

Suppose $x<y<\alpha$, we have $LHS = 0\le |x-y|$.

These verified this statement.

### EXERCISE 1.3.2

Show that 

$$
g(h) = c+ \beta\sum_{w'}\max\left\{\frac{w'}{1-\beta},h\right\}\varphi(w')
$$

is a contraction map on $\mathbb{R}_+$. Conclude that $h^*$ is the unique fixed point of $g$ in $\mathbb{R}_+$.

**Proof**

Let $h_1<h_2\in \mathbb{R}_+$. We have,

$$
\|g(h_1)-g(h_2)\| = \beta\left|\sum_{w'}\left(\max\left\{\frac{w'}{1-\beta},h_1\right\}-\max\left\{\frac{w'}{1-\beta},h_2\right\}\right)\varphi(w')\right|
$$

Using the result from Exercise 1.3.1, we have

$$
\|g(h_1)-g(h_2)\| \le \beta\left|\sum_{w'}\left(h_1-h_2\right)\varphi(w')\right|=\beta(h_1-h_2)
$$

Hence, $g$ is a contraction on $\mathbb{R}_+$.

Moreover, since $\mathbb{R}_+$ is complete. This fixed point is unique.

### EXERCISE 1.3.3 

Compare the value function $v^*$ computed via

$$
v^*(w) = \max\left\{\frac{w}{1-\beta},h^*\right\}
$$

with previous results.

**Solution**:

We start by constructing the successive approximation function (see details in the reading note).

In [7]:
def successive_approx (
    S,                     # A callable operator
    x_0,                   # Initial condition, the arbitary starting point
    model,                 # Model parameters
    tol = 1e-6,            # Error tolerance used for approximation
    max_iter = 10_000,     # max interation to avoid infinite iteration due to not converging
    print_step = 25        # Print at multiples of print_step
):
    x = x_0                # set the initial condition
    error = tol + 1        # Initialize the error
    k = 1                  # initialize the interations
    
    while (error > tol) and (k <= max_iter): 
        x_new = S(x,model)       # update by applying operator T
        error = np.max(np.abs(x_new-x))  # the valuation of error is based on the norm
        if k % print_step == 0:  # the remainder of k/print_step is zero
            print(f"Completed iteration {k} with error {error}.") # This prints the error value if the steps is divisible by the print_step
        x = x_new         # assign the new value to x
        k += 1            # update the number of step by 1
    if error <= tol:      # After the iteration finishes and if error is small
        print(f"Terminated successfully in {k} interations.")
    else:     
        print("Warning: hit iteration bound.")
    return x              # if successful, x is the fixed point

Define the Bellman operator as follows

In [8]:
def S (
    v,        # the value function
    model     # model parameters
):
    c, w_vals, n, β, T = model.c, model.w_vals, model.n, model.β, model.T  # Input the model parameters
    return np.maximum(w_vals/(1-β), c + β * np.mean(v))                    # return the value as state before

Then construct the $v$-greedy policy

In [9]:
def get_greedy (
    v,        # the value function, a np array
    model     # model parameters
): 
    c, w_vals, n, β, T = model.c, model.w_vals, model.n, model.β, model.T  # Input the model parameters
    σ= np.where(w_vals/(1-β) >= c + β * np.mean(v), 1, 0)                  # v-greedy
    return σ

Now we use value function iteration method to obtain the value function $v^*$ and optimal policy/$v^*$-greedy policy $\sigma^*$

In [10]:
def value_function_iteration (model):
    c, w_vals, n, β, T = model.c, model.w_vals, model.n, model.β, model.T  # Input the model parameters
    v_init = np.zeros(n) # initialize the guess of value function
    v_star = successive_approx(S, v_init, model)
    σ_star = get_greedy(v_star, model)
    
    return v_star, σ_star

In [11]:
value_function_iteration(model)

Completed iteration 25 with error 0.03455934557837281.
Completed iteration 50 with error 6.7009955273533706e-06.
Terminated successfully in 57 interations.


(array([1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1200.        , 1225.        , 1250.        ,
        1275.        , 1300.        , 1325.        , 1350.        ,
        1375.        , 1400.        , 1425.        , 1450.        ,
        1475.        , 1500.        ]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

This shows the result that using value function iteration.

**New Algorithm**

1. compute $h^*$ via successive approximation on $g$ (iterate in $\mathbb{R}$ not in $\mathbb{R}^n$)

2. Optimal policy is 

$$
\sigma^*(w) = \mathbb{1}\left\{\frac{w}{1-\beta}\ge h^*\right\}
$$

We first define this operator $g$.

In [12]:
def g (
    h,      #input reservation wage
    model   # Model parameters
):
    c, w_vals, n, β, T = model.c, model.w_vals, model.n, model.β, model.T  # Input the model parameters
    return c + β * np.mean(np.maximum(w_vals/(1-β), h))

Now we use the second method,

In [13]:
def direct_iteration (model):
    c, w_vals, n, β, T = model.c, model.w_vals, model.n, model.β, model.T  # Input the model parameters
    h_init = 0    # inital guess
    h_star = successive_approx(g, h_init, model)  # succesive approximation on g
    v_star = np.maximum(w_vals/(1-β),h_star)
    return v_star, h_star

In [14]:
direct_iteration(model)

Completed iteration 25 with error 0.024550959099087777.
Completed iteration 50 with error 4.7603871280443855e-06.
Terminated successfully in 56 interations.


(array([1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1198.06629623, 1198.06629623, 1198.06629623,
        1198.06629623, 1200.        , 1225.        , 1250.        ,
        1275.        , 1300.        , 1325.        , 1350.        ,
        1375.        , 1400.        , 1425.        , 1450.        ,
        1475.        , 1500.        ]),
 1198.0662962297258)

This shows we obtain the same result.